In [11]:
from IPython.core.interactiveshell import InteractiveShell  #执行该代码可以使得当前nb支持多输出
InteractiveShell.ast_node_interactivity = "all" 

In [12]:
import requests
from bs4 import BeautifulSoup
from lxml import etree
import pandas as pd
from fake_useragent import UserAgent
import time

In [227]:
shops = pd.read_excel('店铺列表3.xlsx')
shops.iloc[:,1]
shops.iloc[:,0]

0     2200650694048
1     2200640617541
2        3922061057
3        3901445305
4        3837586220
          ...      
74    2210228542894
75       3014691900
76       3010242861
77        720439558
78        382051931
Name: 店铺id, Length: 79, dtype: int64

0      喵喵的衣服shop
1      花花公子贵宾特价店
2     富贵鸟纯棉世家男装店
3       布丁原创shop
4       右米家女装批发店
         ...    
74        喜儿品质服饰
75    运动三叶草品牌折扣店
76       峻哲男装服饰店
77     阿予家精品女装批发
78        媛妃琪の服饰
Name: 店铺名称, Length: 79, dtype: object

In [228]:
url_shop = 'https://www.dianchacha.com/shop/info/index/uid/'
url_serch = 'https://www.dianchacha.com/shop/keyword/index/uid/'
url_shop = [url_shop + '{}'.format(i) for i in shops.iloc[:,1]]
url_serch = [url_serch + '{}'.format(i) +'?p=5' for i in shops.iloc[:,1]]

In [191]:
rate = ['指标','上周同期','较上周']
def get_sales(url,j):
    resp = requests.get(url,headers = headers)
    html = resp.content.decode('utf-8')
    e_html = etree.HTML(html)
    df1=pd.read_html(html)[0]
    rate = ['指标','上周同期','较上周']    
    if('没有人关注' not in df1.iloc[0,0]):
        for i in range(1,8):
            df1[df1.columns[i]+rate[0]] = df1.iloc[:,i].map(lambda x:x.split(' ')[0])
            df1[df1.columns[i]+rate[1]] = df1.iloc[:,i].map(lambda x:x.split(' ')[1])
            df1[df1.columns[i]+rate[2]] = df1.iloc[:,i].map(lambda x:x.split(' ')[2])
        df1.index=pd.MultiIndex.from_product([[shops.iloc[j,0]],['销售额（元）','销售量（件）','宝贝数（款）']])
        df1 = df1.drop(df1.iloc[:,:8],axis=1)
    return df1

In [206]:
def get_hot(url,j):
    resp2 = requests.get(url,headers = headers)
    html2 = resp2.content.decode('utf-8')
    e_html2 = etree.HTML(html2)
    df2=pd.read_html(html2)[0]
    df2 = df2.drop(['Unnamed: 0','操作'],axis = 1)
    df2.index = [[shops.iloc[j,0]]*len(df2)]
    return df2

In [64]:
#基础信息
def get_shopDetails(url1,url2):
    resp = requests.get(url1,headers = headers)
    html = resp.content.decode('utf-8')
    e_html1 = etree.HTML(html)
    resp2 = requests.get(url2,headers = headers)
    html2 = resp2.content.decode('utf-8')
    e_html2 = etree.HTML(html2)
    #获取开店时间
    open_time = e_html1.xpath('/html/body/div[2]/div[2]/div[2]/div[1]/ul[1]//text()')
    #获取店铺信用
    credits = e_html1.xpath('/html/body/div[2]/div[2]/div[2]/div[1]/ul[1]/li[1]/div[2]/span[2]/@title')
    sale = e_html2.xpath('/html/body/div[2]/div[2]/div[2]/div/div//text()')
    info = []
    info.append(open_time)
    info.append(sale)
    result= []
    result_dict = {}
    for lists in info:
        for i in lists:
            i = i.replace('\n','')
            i = i.replace(' ','')
            if('：' in i):
                i = i.replace('：',':')
            result.append(i)
            while '' in result:
                result.remove('')
    for i in range(len(result)):
        if(':' in result[i]):
            result[i] = result[i].replace(':','')
            result_dict[result[i]] = result[i+1]

    result_dict['卖家信用'] = credits[0]
    result_df = pd.DataFrame(result_dict,index=['1'])
    result_df = result_df.set_index(['店铺'])
    return result_df

In [229]:
ua = UserAgent().random
cookie = '_dcc_session=iq51o4ab7o1n6rqf2td2dhj132; Hm_lvt_2dc49552d570922026e5fcb79894c7b3=1624680783,1624718399,1624761303,1624776810; Hm_lvt_3a75bcb07225c9d03aae2d67edca6226=1624680783,1624718399,1624761303,1624776810; acw_tc=76b20f4516247907602958208e9a4d984d832edcb40f455fcf5513f31b1dd2; _sess_remember=1e03fc9f06a7e9aed4521b37a914f7df; Hm_lpvt_3a75bcb07225c9d03aae2d67edca6226='+str(int(time.time()))+'; Hm_lpvt_2dc49552d570922026e5fcb79894c7b3='+str(int(time.time()))
headers = {
    'Accept': '*/*',
    'Accept-Language': 'zh-CN,zh;q=0.9',
    'Cache-Control': 'max-age=0',
    'Cookie':cookie,
    'User-Agent':ua,
    'Connection': 'keep-alive',
    'Referer': 'https://www.dianchacha.com/shop/info/index/uid'
}
for i in range(79):
    sales = pd.concat([sales,get_hot(url_serch[i],i)])
    time.sleep(10)